In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
from timer import Timer
import pickle

In [6]:
neuronMax = 1
neuronMin = -1
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0)
        return np.maximum(0, x)

    @staticmethod
    def leakyRelu(x, derivative=False):
        if derivative:
            return 1.0 * (x > 0) + 0.01 * (x <= 0)
        return np.maximum(0.01 * x, x)

    @staticmethod
    def absolute(x, derivative=False):
        if derivative:
            return 1 if x > 0 else -1
        return np.abs(x)


    # @staticmethod
    # def softmax(x):
    #     exps = np.exp(x - np.max(x))
    #     return exps / np.sum(exps, axis=1, keepdims=True)

    # @staticmethod
    # def softmaxDerivative(x):
    #     return x * (1 - x)
    @staticmethod
    def mseLoss(errorVector, derivative=False):
        if derivative:
            return errorVector
        return np.mean(np.square(errorVector))*0.5

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )



    def backpropagation(self, target, learning_rate, lossFunction):
        output_error = lossFunction(target - self.layers[-1], derivative=True)

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error * derivative, self.weights[i].T) * learning_rate

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.outer(self.layers[i], output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += weight_adjustments * learning_rate
            self.biases[i] += bias_adjustments * learning_rate

            # Set the output error for the next iteration
            # np.normalize(layer_error)
            output_error = layer_error


In [18]:
x = NeuralNetwork([5_000, 40, 40, 169], [NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu, NeuralNetwork.leakyRelu])
nnTimer = Timer(precision=3)
# target = np.arange(169).astype(neuronDtype)
target = np.random.uniform(low = -1000, high = 1000, size = 169)
input = np.random.uniform(low = 0, high = 1, size = 5_000)

In [19]:
x.loadInput(input)
x.forward()
print(x.readOutput())

[-6.77138984e-01  2.39950958e+02  5.05083351e+01  7.46223164e+00
 -2.40959376e-01  1.26209114e+02  9.14057636e+00 -1.94085181e+00
  4.47919312e+01  1.29060450e+01 -4.84230593e-02 -2.16366577e+00
  1.94475235e+02 -9.70843136e-02  2.27592361e+02  5.67448273e+01
 -2.71036357e-01  1.70890076e+02  2.58556152e+02 -1.10078180e+00
  2.37186188e+02 -8.46472979e-01 -1.84588182e+00  1.87487183e+01
 -1.48240328e+00  4.60992966e+01  1.27058449e+02 -6.79023862e-01
  1.68981247e+02 -3.84860665e-01 -3.38178098e-01  1.96714828e+02
  1.34315948e+02 -1.60266664e-02 -4.21629757e-01 -5.00999272e-01
  9.13215408e+01 -8.47731769e-01 -1.89133143e+00  4.49798737e+01
 -4.29648161e-01 -3.28143826e-03 -2.76518047e-01  2.13599487e+02
  2.17311752e+02 -7.09969163e-01  2.81728191e+01 -2.15708479e-01
  3.05834290e+02 -4.33292180e-01 -1.74448359e+00  2.49237030e+02
 -1.08326745e+00 -8.89153183e-01 -9.74337816e-01  2.93538799e+01
 -2.55043268e+00 -1.77460873e+00 -1.04417908e+00 -1.16088223e+00
  1.48191544e+02  1.56172

In [20]:
# learning_rate = 0.00001
# for i in range(200):
#     nnTimer.tic()
#     x.loadInput(input)
#     x.forward()
#     x.backpropagation(target, learning_rate)
#     print(x.readOutput())
#     nnTimer.toc()
# nnTimer.stats()
learning_rate = 0.00001
for i in range(2000):
    nnTimer.tic()
    x.loadInput(input)
    x.forward()
    x.backpropagation(target, learning_rate, NeuralNetwork.mseLoss)
    print(x.readOutput(), NeuralNetwork.mseLoss(target - x.readOutput()))
    nnTimer.toc()
nnTimer.stats()

[-6.77138984e-01  2.39950958e+02  5.05083351e+01  7.46223164e+00
 -2.40959376e-01  1.26209114e+02  9.14057636e+00 -1.94085181e+00
  4.47919312e+01  1.29060450e+01 -4.84230593e-02 -2.16366577e+00
  1.94475235e+02 -9.70843136e-02  2.27592361e+02  5.67448273e+01
 -2.71036357e-01  1.70890076e+02  2.58556152e+02 -1.10078180e+00
  2.37186188e+02 -8.46472979e-01 -1.84588182e+00  1.87487183e+01
 -1.48240328e+00  4.60992966e+01  1.27058449e+02 -6.79023862e-01
  1.68981247e+02 -3.84860665e-01 -3.38178098e-01  1.96714828e+02
  1.34315948e+02 -1.60266664e-02 -4.21629757e-01 -5.00999272e-01
  9.13215408e+01 -8.47731769e-01 -1.89133143e+00  4.49798737e+01
 -4.29648161e-01 -3.28143826e-03 -2.76518047e-01  2.13599487e+02
  2.17311752e+02 -7.09969163e-01  2.81728191e+01 -2.15708479e-01
  3.05834290e+02 -4.33292180e-01 -1.74448359e+00  2.49237030e+02
 -1.08326745e+00 -8.89153183e-01 -9.74337816e-01  2.93538799e+01
 -2.55043268e+00 -1.77460873e+00 -1.04417908e+00 -1.16088223e+00
  1.48191544e+02  1.56172

In [24]:
predictions = x.readOutput()
for i in range(len(predictions)):
    print(f"Prediction:     {predictions[i]},               Target:    {target[i]}, Iteration:    {i}")

mse = target - predictions
sq = sum(mse**2)/len(mse)
np.sqrt(sq)

106.27923002494586

In [14]:
import numpy as np

size = 5
padding = 3

a = np.zeros((size + padding * 2, size + padding * 2))

for i in range(padding, size + padding):
    for j in range(padding, size + padding):
        a[i, j] = i - padding + (j - padding) * 10

In [15]:
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 10., 20., 30., 40.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., 11., 21., 31., 41.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  2., 12., 22., 32., 42.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  3., 13., 23., 33., 43.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4., 14., 24., 34., 44.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])